# Title: Python Series – Day 34: Decorators in Python (Advanced)

## 1. Introduction
**Decorators** are a powerful and advanced feature in Python that allows you to modify the behavior of a function or class without permanently changing it.

**Why use Decorators?**
- **Extensibility:** Add functionality (like logging, timing) to existing functions.
- **Reusability:** Write logic once and apply it to many functions.
- **Clean Code:** Separate core logic from auxiliary concerns (Separation of Concerns).

**Real-world uses:**
- Logging API calls.
- Authentication/Permission checks.
- Caching results.
- Measuring execution time.

## 2. Functions as First-Class Citizens
Before understanding decorators, remember that in Python:
1. Functions can be assigned to variables.
2. Functions can be passed as arguments.
3. Functions can return other functions.

In [ ]:
def shout(text):
    return text.upper()

# Assign to variable
yell = shout
print(yell("hello"))

# Pass as argument
def greet(func):
    print(func("hi there"))

greet(shout)

## 3. Understanding Decorator Basics
A decorator is just a function that takes another function as an argument, adds some functionality, and returns a new function.

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

# Calling the decorated function
say_hello()

## 4. Decorators with Arguments
To decorate functions that accept arguments, the wrapper function must accept them too.

In [ ]:
def log_args(func):
    def wrapper(x):
        print(f"Calling function with argument: {x}")
        return func(x)
    return wrapper

@log_args
def square(n):
    return n * n

print(f"Result: {square(5)}")

## 5. Decorators with *args & **kwargs
To make a decorator work with any function signature, use `*args` and `**kwargs`.

In [ ]:
def dynamic_logger(func):
    def wrapper(*args, **kwargs):
        print(f"LOG: Running {func.__name__} with args={args} kwargs={kwargs}")
        return func(*args, **kwargs)
    return wrapper

@dynamic_logger
def add(a, b):
    return a + b

@dynamic_logger
def introduce(name, age=0):
    print(f"I am {name}, {age} years old.")

print(add(10, 20))
introduce("Ali", age=25)

## 6. Returning Values from Decorators
The wrapper function **must return** the result of the original function call, otherwise the return value will be lost (return `None`).

In [ ]:
def ensure_return(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"Function returned: {result}")
        return result # Crucial!
    return wrapper

@ensure_return
def multiply(a, b):
    return a * b

val = multiply(3, 4)
print(f"Value in main: {val}")

## 7. Practical Use Cases

### 7.1 Timer Decorator
Measure how long a function takes to run.

In [ ]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

@timer
def slow_function():
    time.sleep(1)
    print("Finished sleeping")

slow_function()

## 8. Decorators with Parameters (Decorator Factory)
If you want to pass arguments to the decorator itself (e.g., `@repeat(3)`), you need a three-layer function.

In [ ]:
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(times):
                print(f"Run {i+1}:")
                func(*args, **kwargs)
        return wrapper
    return decorator

@repeat(3)
def greet():
    print("Hello World")

greet()

## 9. Using functools.wraps
When decorating, the original function's metadata (name, docstring) gets lost. Use `@wraps` from `functools` to fix this.

In [ ]:
from functools import wraps

def my_decorator(func):
    @wraps(func) # Preserves func metadata
    def wrapper(*args, **kwargs):
        """Wrapper function docstring"""
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_func():
    """Original function docstring"""
    pass

print(f"Function name: {my_func.__name__}")
print(f"Docstring: {my_func.__doc__}")

## 10. Class-Based Decorators
Decorators can also be classes by implementing the `__call__` method.

In [ ]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hi():
    print("Hi!")

say_hi()
say_hi()

## 11. Practice Exercises
1. Create `logging_decorator` that prints function name and time of execution.
2. Create `admin_required` decorator (simulate user object `{"admin": False}`).
3. Create a decorator that forces the function to run only if the first argument is positive.
4. Create a decorator `uppercase_result` that converts return value string to uppercase.
5. Create a `@repeat(n)` decorator factory.

## 12. Mini Project – API Call Logger
Log details of every API call made by a function.

In [ ]:
import requests
import datetime

def api_logger(func):
    def wrapper(*args, **kwargs):
        # Extract URL (assuming it's the first arg)
        url = args[0] if args else "Unknown URL"
        start = datetime.datetime.now()
        
        try:
            response = func(*args, **kwargs)
            status = response.status_code
        except Exception as e:
            status = "ERROR"
            raise e
        finally:
            # Log to file
            with open("api_logs.txt", "a") as f:
                log_entry = f"[{start}] URL: {url} | Status: {status}\n"
                f.write(log_entry)
                print(f"Logged: {log_entry.strip()}")
        
        return response
    return wrapper

@api_logger
def fetch_data(url):
    return requests.get(url)

# Test logger
try:
    fetch_data("https://jsonplaceholder.typicode.com/users")
    fetch_data("https://jsonplaceholder.typicode.com/posts/1")
except Exception as e:
    print(e)

## 13. Day 34 Summary
- **Decorators**: Modify behavior without changing source code.
- **Wrapper**: The inner function that `*args` and `**kwargs`.
- **Returning Values**: Essential for correct function behavior.
- **Factories**: Decorators handling arguments.
- **@wraps**: Preserving metadata.

**Next topic: Day 35 – Python Context Managers**